<a href="https://colab.research.google.com/github/skycreeds/LLM_SERVER/blob/master/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

In [ ]:
!ngrok config add-authtoken 2bb4ZJdDB6G3hofGVVv7mtxdsHY_2jGXBAH6YYYKqQNhNXJLm

In [ ]:
!pip install langchain bitsandbytes sentence-transformers faiss-cpu playwright html2text

In [ ]:
! playwright install

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import Html2TextTransformer
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import langchain_community
import nest_asyncio
nest_asyncio.apply()

In [ ]:
hug_embed=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

In [ ]:
def create_or_add_docs(docs:list,index:langchain_community.vectorstores.faiss.FAISS = None):
  loader = AsyncChromiumLoader(docs)
  docs = loader.load()
  html2text = Html2TextTransformer()
  docs_transformed = html2text.transform_documents(docs)
  text_splitter = CharacterTextSplitter(chunk_size=100,
                                      chunk_overlap=0)
  chunked_documents = text_splitter.split_documents(docs_transformed)
  if index is None:
    db = FAISS.from_documents(chunked_documents,
                          hug_embed)

  return db

In [ ]:
db=None

In [ ]:
from fastapi import FastAPI,UploadFile
from fastapi.middleware.cors import CORSMiddleware
import json

app = FastAPI()


@app.post("/link")
async def receive_data(link:dict):
    print(link["abc"])
    result = link["abc"].split(',')
    db=create_or_add_docs(result)
    return result


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn


# specify a port
port = 8000
ngrok_tunnel = ngrok.connect(port)

print('Public URL:', ngrok_tunnel.public_url)


nest_asyncio.apply()

# finally run the app
uvicorn.run(app, port=port)